In [49]:
import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [50]:
import os
path_working_directory = os.getcwd()

educacion = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_1_postulantes_educacion.csv"), low_memory=False)
genero_edad = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_2_postulantes_genero_y_edad.csv"), low_memory=False)
visitas = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_3_vistas.csv"), low_memory=False)
postulaciones = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_4_postulaciones.csv"), low_memory=False)
avisos_online = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_5_avisos_online.csv"), low_memory=False)
avisos_detalles = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_6_avisos_detalle.csv"), low_memory=False)

In [13]:
import calendar
print len(genero_edad), '\n'
print genero_edad.isnull().any(), '\n'
print genero_edad.isnull().sum(), '\n'
print genero_edad.dtypes
no_declaran_edad = genero_edad[genero_edad['fechanacimiento'].isnull()]
print len(no_declaran_edad), '\n'
postulantes_sin_fecha_nacimiento = pd.merge(no_declaran_edad, postulaciones, how='inner', on='idpostulante')
print "Postulaciones de usuarios sin fecha de nacimiento", len(postulantes_sin_fecha_nacimiento), "de", len(postulaciones), '\n'
visitas_sin_fecha_nacimiento = pd.merge(no_declaran_edad, visitas, how='inner', on='idpostulante')
print "Visitas de usuarios sin fecha de nacimiento", len(visitas_sin_fecha_nacimiento), "de", len(visitas)

200888 

idpostulante       False
fechanacimiento     True
sexo               False
dtype: bool 

idpostulante          0
fechanacimiento    4750
sexo                  0
dtype: int64 

idpostulante       object
fechanacimiento    object
sexo               object
dtype: object
4750 

Postulaciones de usuarios sin fecha de nacimiento 44464 de 3401623 

Visitas de usuarios sin fecha de nacimiento 22767 de 961897


In [4]:
print len(visitas), '\n'
print visitas.isnull().any(), '\n'
print visitas.dtypes

961897 

idAviso         False
timestamp       False
idpostulante    False
dtype: bool 

idAviso          int64
timestamp       object
idpostulante    object
dtype: object


In [5]:
print len(postulaciones), '\n'
print postulaciones.isnull().any(), '\n'
print postulaciones.dtypes

3401623 

idaviso             False
idpostulante        False
fechapostulacion    False
dtype: bool 

idaviso              int64
idpostulante        object
fechapostulacion    object
dtype: object


In [6]:
print len(avisos_online), '\n'
print avisos_online.isnull().any(), '\n'
print avisos_online.dtypes

5028 

idaviso    False
dtype: bool 

idaviso    int64
dtype: object


In [7]:
print len(avisos_detalles), '\n'
print 'Columnas con valores nulos:'
print avisos_detalles.isnull().any(), '\n'
print 'Cantidad de valores nulos por columna'
print avisos_detalles.isnull().sum(), '\n'
print avisos_detalles.dtypes

13534 

Columnas con valores nulos:
idaviso                 False
idpais                  False
titulo                  False
descripcion             False
nombre_zona             False
ciudad                   True
mapacalle                True
tipo_de_trabajo         False
nivel_laboral           False
nombre_area             False
denominacion_empresa     True
dtype: bool 

Cantidad de valores nulos por columna
idaviso                     0
idpais                      0
titulo                      0
descripcion                 0
nombre_zona                 0
ciudad                  13487
mapacalle               12662
tipo_de_trabajo             0
nivel_laboral               0
nombre_area                 0
denominacion_empresa        5
dtype: int64 

idaviso                  int64
idpais                   int64
titulo                  object
descripcion             object
nombre_zona             object
ciudad                  object
mapacalle               object
tipo_de_trabajo     

In [8]:
print educacion.isnull().any(), '\n'
print educacion.describe(include='all'), '\n'
print educacion.dtypes, '\n'

idpostulante    False
nombre          False
estado          False
dtype: bool 

       idpostulante      nombre    estado
count        298231      298231    298231
unique       188752           7         3
top          YlMLGD  Secundario  Graduado
freq              9      110256    194474 

idpostulante    object
nombre          object
estado          object
dtype: object 



In [51]:
#Como se puede observar, algunos postulantes tienen mas de una entrada de educacion
#Por ello, crearemos un nuevo dataframe que tenga una entrada por postulante en el 
#cual las columnas seran idpostulante, en_curso y graduado. En las columnas en_curso
#y graduado se indicara el estudio mas importante que se encuentra (o encontraba, dado
#que no podemos saber si el mismo sigue o no en ese estado) en el estado indicado por
#el nombre de la columna
# print educacion.nombre.value_counts(), '\n'
# print educacion.estado.value_counts(), '\n'
# tipos_de_estudio = ['', 'Otro', 'Secundario', 'Terciario/Técnico', 'Universitario', 'Master', 'Posgrado', 'Doctorado']
# def get_dict_tipos_de_estudio():
#     dict = {}
#     for x in range(len(tipos_de_estudio)):
#         dict[tipos_de_estudio[x]] = x
#     return dict
# dict_tipos_de_estudio = get_dict_tipos_de_estudio()
# def mejor_estudio(estudio1, estudio2):
#     return tipos_de_estudio[max(tipos_de_estudio.index(estudio1), tipos_de_estudio.index(estudio2))]

# en_curso = educacion.apply(lambda x: x['nombre'] if x['estado'] == 'En Curso' else '', axis=1)
# graduado = educacion.apply(lambda x: x['nombre'] if x['estado'] == 'Graduado' else '', axis=1)
# educacion2 = educacion.assign(estudio_en_curso=en_curso)
# educacion2 = educacion2.assign(estudio_graduado=graduado)
# educacion2[educacion2['estudio_graduado'] != '']
# educacion2.groupby('idpostulante').get_group('YlMLGD')
# educacion_groups = educacion2.groupby('idpostulante')
# educacion_groups = educacion_groups['estudio_en_curso'].unique()
# # # educacion_groups['estudio_en_curso'].unique()
# educacion_groups.reset_index(inplace=True)

In [58]:
tipos_de_estudio = ['', 'Otro', 'Secundario', 'Terciario/Técnico', 'Universitario', 'Master', 'Posgrado', 'Doctorado']
def get_dict_tipos_de_estudio():
    dict = {}
    for x in range(len(tipos_de_estudio)):
        dict[tipos_de_estudio[x]] = x
    return dict

dict_tipos_de_estudio = get_dict_tipos_de_estudio()

def mejor_estudio(estudio1, estudio2):
    return tipos_de_estudio[max(tipos_de_estudio.index(estudio1), tipos_de_estudio.index(estudio2))]

en_curso = educacion.apply(lambda x: dict_tipos_de_estudio[x['nombre']] if x['estado'] == 'En Curso' else 0, axis=1)
graduado = educacion.apply(lambda x: dict_tipos_de_estudio[x['nombre']] if x['estado'] == 'Graduado' else 0, axis=1)
educacion2 = educacion.assign(estudio_en_curso=en_curso)
educacion2 = educacion2.assign(estudio_graduado=graduado)

In [69]:
educacion_grupos = educacion2.groupby('idpostulante')
educacion_nueva = pd.DataFrame({
    'estudio_en_curso': educacion_grupos['estudio_en_curso'].max(),
    'estudio_graduado': educacion_grupos['estudio_graduado'].max()
}).reset_index()
educacion_nueva['estudio_en_curso'] = educacion_nueva['estudio_en_curso'].apply(lambda x: tipos_de_estudio[x])
educacion_nueva['estudio_graduado'] = educacion_nueva['estudio_graduado'].apply(lambda x: tipos_de_estudio[x])
educacion_nueva

,idpostulante,estudio_en_curso,estudio_graduado
0,0z5VvGv,,Secundario
1,0zB026d,,Terciario/Técnico
2,0zB03O9,Terciario/Técnico,Secundario
3,0zB08lq,,Secundario
4,0zB0NWY,,Universitario
5,0zB0Xwd,Universitario,Secundario
6,0zB0YGv,Otro,
7,0zB0e1d,Universitario,Secundario
8,0zB0eDY,,Terciario/Técnico
9,0zB0qGE,,Secundario
